<a href="https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/06PyTorchTEXT_torchtext_translation_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
    
このファイルは PyTorch のチュートリアルにあるファイル <https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html> を翻訳して，加筆修正したもの
です。

すぐれたチュートリアルの内容，コードを公開された Ben Trevett, Seth Widman および PyTorch 開発陣に敬意を表します。

- Original: 
- Date: 2020-0811
- Translated and modified: Shin Asakawa <asakawa@ieee.org>

---

In [7]:
# 2020年8月11日現在 torchtext を upgrade しないとこのノートブックは動作しません。
!pip install --upgrade torchtext

     |████████████████████████████████| 4.5MB 7.8MB/s 
     |████████████████████████████████| 1.1MB 55.6MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [1]:
# from https://github.com/dmlc/xgboost/issues/1715
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
%matplotlib inline

# TorchText による言語翻訳
<!--
# Language Translation with TorchText
-->

<!--
This tutorial shows how to use several convenience classes of ``torchtext`` to preprocess data from a well-known dataset containing sentences in both English and German and use it to train a sequence-to-sequence model with attention that can translate German sentences into English.
-->

このチュートリアルでは，英語とドイツ語の両方の文を含むよく知られたデータセットからのデータの前処理し，ドイツ語の文を英語に翻訳できるように注意を使った系列-to-系列 (seq2seq) モデルを訓練します。このため ``torchtext`` のいくつかの便利なクラスを使用する方法を示します。

<!--
It is based off of [this tutorial](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) from PyTorch community member [Ben Trevett](https://github.com/bentrevett) and was created by [Seth Weidman](https://github.com/SethHWeidman/) with Ben's permission.
-->

このチュートリアルは PyTorch コミュニティメンバーの [Ben Trevett](https://github.com/bentrevett) の [チュートリアル](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) をベースにしています。
Ben の許可を得て [Seth Weidman](https://github.com/SethHWeidman/) によって作成されました。


このチュートリアルを終えると，以下のことができるようになります。

- 以下 ``torchtext`` 便利なクラスを使って NLP モデリングのために一般的に使われている形式に文を前処理
    - [TranslationDataset](https://torchtext.readthedocs.io/en/latest/datasets.html#torchtext.datasets.TranslationDataset)
    - [Field](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Field)
    - [BucketIterator](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator)

<!--
By the end of this tutorial, you will be able to:

- Preprocess sentences into a commonly-used format for NLP modeling using the following ``torchtext`` convenience classes:
    - [TranslationDataset](https://torchtext.readthedocs.io/en/latest/datasets.html#torchtext.datasets.TranslationDataset)
    - [Field](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Field)
    - [BucketIterator](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator)
-->


<!-- ## `Field` and `TranslationDataset`-->
## `Field` と `TranslationDataset`

<!--
``torchtext`` has utilities for creating datasets that can be easily iterated through for the purposes of creating a language translation model. 
One key class is a [Field](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L64), which specifies the way each sentence should be preprocessed, and another is the `TranslationDataset` ; ``torchtext`` has several such datasets; in this tutorial we'll use the [Multi30k dataset](https://github.com/multi30k/dataset), which contains about 30,000 sentences (averaging about 13 words in length) in both English and German.
-->

``torchtext`` には，言語翻訳モデルを作成する目的で簡単に反復処理できるデータセットを作成するためのユーティリティが用意されています。
キーとなるクラスの一つは [Field](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L64) です。
Field を用いて各文をどのように前処理するかを指定することができます。

<!--
Note: the tokenization in this tutorial requires [Spacy](https://spacy.io).
We use Spacy because it provides strong support for tokenization in languages other than English. 
``torchtext`` provides a ``basic_english`` tokenizer and supports other tokenizers for English (e.g. [Moses](https://bitbucket.org/luismsgomes/mosestokenizer/src/default/) but for language translation - where multiple languages are required - Spacy is your best bet. 
-->

注意: このチュートリアルのトークン化には [Spacy](https://spacy.io) が必要です。
Spacy を使うのは，英語以外の言語でのトークン化を強力にサポートしているからです。
``torchtext`` は ``basic_english`` のトークン化機能を提供しています。
他のトークン化機能もサポートしています
(例: [Moses](https://bitbucket.org/luismsgomes/mosestokenizer/src/default/) 
複数の言語翻訳が必要な場合には Spacy を使うのが一番です。

To run this tutorial, first install ``spacy`` using ``pip`` or ``conda``. 
Next, download the raw data for the English and German Spacy tokenizers:

このチュートリアルを実行するには まず ``spacy`` を ``pip`` または ``conda`` を使ってインストールします。
次に，英語とドイツ語の Spacy トークナイザーで使用する生データをダウンロードします。

```bash
$python -m spacy download en
$python -m spacy download de
```

<!--
With Spacy installed, the following code will tokenize each of the sentences in the ``TranslationDataset`` based on the tokenizer defined in the ``Field``
-->

Spacy をインストールすると，以下のコードは ``Field`` で定義されたトークン化方式に基づいて ``TranslationDataset`` の各文をトークン化します。



In [3]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),
                                                    fields = (SRC, TRG))

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
training.tar.gz:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.67MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.68MB/s]

downloading validation.tar.gz
downloading mmt_task1_test2016.tar.gz



mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.36MB/s]
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


<!--
Now that we've defined ``train_data``, we can see an extremely useful feature of ``torchtext``'s ``Field``: the ``build_vocab`` method now allows us to create the vocabulary associated with each language
-->
これで ``train_data`` が定義できたので ``torchtext`` の ``Field`` の非常に便利な機能を見ることができます。



In [5]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

<!--
Once these lines of code have been run, ``SRC.vocab.stoi`` will  be a dictionary with the tokens in the vocabulary as keys and their corresponding indices as values; 
``SRC.vocab.itos`` will be the same dictionary with the keys and values swapped. 
We won't make extensive use of this fact in this tutorial, but this will likely be useful in other NLP tasks you'll encounter.
-->

これらのコードを実行すると、 ``SRC.vocab.stoi`` は語彙のトークンをキーとし，それに対応するインデックスを値として持つ辞書になります。
``SRC.vocab.itos`` は同じ辞書で，キーと値が入れ替わっています。
このチュートリアルではこの事実を大々的に利用することはありませんが、他の NLP 課題で遭遇することには役立つでしょう。



## ``BucketIterator``
<!--
The last ``torchtext`` specific feature we'll use is the ``BucketIterator``, which is easy to use since it takes a ``TranslationDataset`` as its first argument. 
Specifically, as the docs say:
Defines an iterator that batches examples of similar lengths together.
Minimizes amount of padding needed while producing freshly shuffled batches for each new epoch. See pool for the bucketing procedure used.
-->

最後に ``torchtext`` に特有の機能として ``BucketIterator`` を使うことにしました。
具体的には、ドキュメントに書かれている通りです:
似たような長さの例をまとめるイテレータを定義します。
新しいエポックごとに新しくシャッフルされたバッチを生成し，必要とされるパディングの量を最小限に抑えます。
使用されるバケット処理については pool を参照してください。



In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


<!--
These iterators can be called just like ``DataLoader``s; below, in the ``train`` and ``evaluate`` functions, they are called simply with:
-->

これらのイテレータは ``DataLoader``s と同様に呼び出すことができます; 
以下では ``train`` と ``evaluate`` 関数の中で、単に ``train`` と ``evaluate`` を使って呼び出されます。

```python
   for i, batch in enumerate(iterator):
```

<!--Each ``batch`` then has ``src`` and ``trg`` attributes:-->
各 ``batch`` は ``src`` と ``trg`` 属性を持ちます:

```python
   src = batch.src
   trg = batch.trg
```


<!--
## Defining our ``nn.Module`` and ``Optimizer``
-->

## ``nn.Module`` と ``Optimizer`` の定義

<!--
That's mostly it from a ``torchtext`` perspecive: with the dataset built and the iterator defined, the rest of this tutorial simply defines our model as an ``nn.Module``, along with an ``Optimizer``, and then trains it.
-->

これが ``torchtext`` の観点から見た場合の大部分です。
このチュートリアルの残りの部分では，構築したデータセットと定義したイテレータを用いることで，ただモデルを ``nn.Module`` と ``Optimizer`` とを使って ``nn.Module`` として定義し、学習するだけです。

ここでのモデルは，[Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) で提案されたアーキテクチャを採用しています。
更に [このノートブック](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) も参照してください)

<!--
Our model specifically, follows the architecture described [here](https://arxiv.org/abs/1409.0473) (you can find a significantly more commented version [here](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb)).
-->

#### 覚書:
<!--#### Note: -->
<!--
this model is just an example model that can be used for language translation; 
we choose it because it is a standard model for the task, not because it is the recommended model to use for translation. 
As you're  likely aware, state-of-the-art models are currently based on Transformers; 
you can see PyTorch's capabilities for implementing Transformer layers [here](https://pytorch.org/docs/stable/nn.html#transformer-layers); and in particular, the "attention" used in the model below is different from the multi-headed self-attention present in a transformer model.
-->

このモデルは，言語翻訳に使用できるモデルの一例に過ぎません。
私たちがこのモデルを選んだのは，この課題の標準モデルだからです。
ですがこのモデルは，翻訳に使用する推奨モデルではありません。
ご存知のように，最先端のモデルは現在 Transformer をベースにしています。
PyTorch の Transformer 層の実装は [こちら](https://pytorch.org/docs/stable/nn.html#transformer-layers) から見ることができます。

特に，以下のモデルで使用されている「注意」は、Transformer モデルでの多頭自己注意とは異なります。


(訳注)
Seq2Seq モデルについては直上セルにあるとおり，Seth H. Weidman のコードが元になっている。
Wiedman のノートブックにあるとおり Seq2Seq モデルのプロトタイプは下図のとおりである

<img src="https://raw.githubusercontent.com/SethHWeidman/pytorch-seq2seq/master/assets/seq2seq1.png" style="width:49%">

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and input word, $y_t$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

前モデルでは、文脈ベクトル $z$ を時間ステップごとに明示的にデコーダに渡し，文脈ベクトルと 入力語 $y_t$ との両方を，隠れ層状態 $s_t$ とともに線形層 $f$ に渡して予測を行うことで，「情報圧縮」を減らすようにアーキテクチャを設定していた。
しかし，このモデルでは，文脈ベクトルと入力語 $y_t$ との両方を 隠れ状態 $s_t$ とともに線形層 $f$ に渡して予測を行うようにした。

<img src="https://raw.githubusercontent.com/SethHWeidman/pytorch-seq2seq/master/assets/seq2seq7.png" style="width:49%">

<!--
Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses attention.
-->

この圧縮を一部削減したとはいえ，文脈ベクトルには原文に関するすべての情報が含まれている必要があります。
このノートブックで実装されているモデルは，デコーダが各デコードステップで（隠れ層の状態を介して）原文全体を見ることを可能にすることで，この圧縮を回避しています。
どうやってこれを行うのでしょうか？それには，注意を持ちています。

<!--Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$.
-->

注意は，まず，原文の長さである注意ベクトル $a$ を計算します。
注意ベクトルは，各要素が 0 から 1 の間にあり，ベクトル全体の和が 1 になるという性質を持っています。
次に、原文の隠れ層状態の加重和 $h$ を計算し，加重された原文ベクトル $w$ を求めます。

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the tutorial.

デコードする際，予測を行う線形層と同様，デコーダ RNN への入力として注意の重みを使用して，タイムステップごとに新しい重み付けされたソースベクトルを計算します。
このチュートリアルでは，これらすべてをどのように行うかを説明します。

---

ここからが本当の訳注: Weidman は上記のように説明していますが，この注意こそがは Bahdanau らの導入した[注意](https://arxiv.org/abs/1409.0473)である。



In [7]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,856,685 trainable parameters


<!--
Note: when scoring the performance of a language translation model in particular, we have to tell the ``nn.CrossEntropyLoss`` function to ignore the indices where the target is simply padding.
-->

覚書: 特に言語翻訳モデルの性能をスコアリングする際には ``nn.CrossEntropyLoss`` 関数に，対象が単にパディングしているだけのインデックスを無視するように指示しなければなりません．



In [8]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

<!---Finally, we can train and evaluate this model:-->
これで，モデル学習と評価ができます



In [ ]:
import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Time: 0m 28s
	Train Loss: 5.687 | Train PPL: 295.101
	 Val. Loss: 5.248 |  Val. PPL: 190.218
Epoch: 02 | Time: 0m 28s
	Train Loss: 5.026 | Train PPL: 152.327
	 Val. Loss: 5.065 |  Val. PPL: 158.323
Epoch: 03 | Time: 0m 29s
	Train Loss: 4.751 | Train PPL: 115.718
	 Val. Loss: 4.960 |  Val. PPL: 142.574
Epoch: 04 | Time: 0m 28s
	Train Loss: 4.592 | Train PPL:  98.737
	 Val. Loss: 5.017 |  Val. PPL: 150.956
Epoch: 05 | Time: 0m 29s
	Train Loss: 4.496 | Train PPL:  89.663
	 Val. Loss: 4.962 |  Val. PPL: 142.874
Epoch: 06 | Time: 0m 29s
	Train Loss: 4.378 | Train PPL:  79.701
	 Val. Loss: 4.966 |  Val. PPL: 143.420


<!--
## Next steps

- Check out the rest of Ben Trevett's tutorials using ``torchtext`` [here](https://github.com/bentrevett/)
- Stay tuned for a tutorial using other ``torchtext`` features along   with ``nn.Transformer`` for language modeling via next word prediction!
-->

## 次のステップ

- Ben Trevett の残りの ``torchtext`` を使ったチュートリアルをチェックしてみてください [https://github.com/bentrevett/](https://github.com/bentrevett/)
- 次単語予測による言語モデリングを用いた，その他の ``torchtext`` の機能と ``nn.Transformer`` を使ったチュートリアルをお楽しみに!


